# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
file = pd.read_csv("../output_data/city.csv")
file.dropna()
file.head()

,City ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,Cape Town,75,ZA,1624610574,77,-33.9258,18.4232,56.12,5.99
1,1,Saskylakh,91,RU,1624610575,50,71.9167,114.0833,70.48,14.45
2,2,Katsuura,100,JP,1624610575,79,35.1333,140.3000,80.11,8.48
3,3,Harper,100,LR,1624610576,85,4.3750,-7.7169,77.54,2.80
4,4,Cherskiy,98,RU,1624610576,52,68.7500,161.3000,54.01,12.86


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
gmaps.configure(api_key=g_key)

In [5]:
locations = file[["Lat", "Lng"]]
humidity = file["Humidity"].astype(float)

In [6]:
# Plot Heatmap
fig = gmaps.figure(center = [0,0] ,zoom_level = 2)

#Create and add heat layer 
heat_layer = gmaps.heatmap_layer(locations, weights=humidity,
                               dissipating=False, max_intensity=100,
                               point_radius = 4)
fig.add_layer(heat_layer)
#Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [9]:
temp_file=file.loc[(file['Max Temp']>=70)&(file['Max Temp']<=80)]
wind_file=temp_file.loc[(temp_file['Wind Speed']<=10)]
ideal_weather=wind_file.loc[(wind_file['Cloudiness']==0)]
ideal_weather.head()

,City ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
143,143,Walker,0,US,1624610661,95,30.7335,-95.5669,78.53,5.50
205,205,Seinäjoki,0,FI,1624610705,60,62.7945,22.8282,76.10,1.01
261,261,Chibuto,0,MZ,1624610739,36,-24.6867,33.5306,76.59,6.87
510,510,Fes,0,MA,1624610890,50,34.0372,-4.9998,79.05,6.91


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [10]:

hotel_df = ideal_weather.reset_index(drop=True)
hotel_df["Hotel Name"] = ""

# params dictionary to update each iteration
params = {
    "radius": 50000,
    "types": "hotel",
    "keyword": "hotels",
    "key": g_key
}

# Use the lat/lng we recovered to identify hotels
for index, row in hotel_df.iterrows():
    
    # get lat, lng 
    lat = row["Lat"]
    lng = row["Lng"]
    
    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"
    
    # Use the search term: "Hotels" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # make request and print url
    hotel_name = requests.get(base_url, params=params)
    
    # convert to json
    hotel_name = hotel_name.json()
    print(json.dumps(hotel_name, indent=4, sort_keys=True))
    
    # Since some data may be missing we incorporate a try-except to skip any that are missing a data point.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotel_name["results"][0]["name"]
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
hotel_df

{
    "html_attributions": [],
    "next_page_token": "Aap_uEBeMo6wFjqiN2F9OmvBTNGFVSjly4M5ha4RvVO2zQQttSA1P5Xz8FFs-OvU2ZvnsM_Tjea1_J7Yy2_reZwASs6vwiai6Bx-hrLiSlUr2UODsL41IYZUDx_vQOHg_PklGxhJbju_KjPwhx9f01ykP85-TccOKf--gwLDcGngJzvNFul-s2xsngbH1H0-nInAOqDJDztKiq_b0QXJo7eN-3lvW9YYCBZBPF-wg45UmXBQWIAjsVFHvx3_X53Wf5q1YJ5AGWPKX9LXscwSa5kMS-NOQpQLppbyp0-R42kW0ZRzUqUA1Aqz9-X1SJ23RJKKQ0sfb9Ozrk4IrH5YvKlyMrH9weudQQn_NIUbVX8sVuVIFNotB1yGB1zyWrTrph8nwMwVQpJvMQYwkIRlAedBtalosN5G2nb13ubTZF8B6vvHWUfZd8OMXCBHczhDBQ2r",
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": 30.8159723,
                    "lng": -95.0929309
                },
                "viewport": {
                    "northeast": {
                        "lat": 30.81701682989272,
                        "lng": -95.09162827010728
                    },
                    "southwest": {
                        "lat": 30.8143171

{
    "html_attributions": [],
    "next_page_token": "Aap_uEDzpClgJdUwdQ4pizsWRwX8V6dog3REwg1xAY3xLYApilwkX2MmJcOY_e1yRdWhfIMQofOm9qgLMJL2zeP429uuM1NZycPcDX2zkPXxTZkluFVuQmfkq0z_RlI1W-cx_JmjOqUgBNo4hjufD4eorR0zYl_1Fklt_RWLgJ8tvBYWSKdhZnSiw0_S5YGsUZPOcc4j8jTNRyzak0lGeFnQLIYqU20zoNyFV1tYIl4ptOL0ANo85wyTfUYbFPKl-5r1n_kTqxxnjZ709oK-pYMbMejEhQ3hjgYAFtV09W1yUEly-KP5PPWCP4vRe1NVwBaEVXe-XdvVjY30vzxEx8lbLkhJe9QfpgKzfO_BMZKx5NJP3gWkLslV7KNXjTqyxCtfkYcK3wYmCKR7R6cqkl62xZv88qEOAHJ8q-_0xuzhCosj4hT4wb1Yn4j1G9P7zZYV",
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": 62.6033287,
                    "lng": 23.7223949
                },
                "viewport": {
                    "northeast": {
                        "lat": 62.60470682989273,
                        "lng": 23.72336737989272
                    },
                    "southwest": {
                        "lat": 62.602007170

{
    "html_attributions": [],
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": -25.1127327,
                    "lng": 33.749096
                },
                "viewport": {
                    "northeast": {
                        "lat": -25.11145582010728,
                        "lng": 33.75048387989272
                    },
                    "southwest": {
                        "lat": -25.11415547989272,
                        "lng": 33.74778422010728
                    }
                }
            },
            "icon": "https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png",
            "name": "Go Wild Beach Resort",
            "opening_hours": {
                "open_now": true
            },
            "photos": [
                {
                    "height": 2269,
                    "html_attributions": [
                        

{
    "error_message": "You must enable Billing on the Google Cloud Project at https://console.cloud.google.com/project/_/billing/enable Learn more at https://developers.google.com/maps/gmp-get-started",
    "html_attributions": [],
    "results": [],
    "status": "REQUEST_DENIED"
}
Missing field/result... skipping.


,City ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
0,143,Walker,0,US,1624610661,95,30.7335,-95.5669,78.53,5.50,Kickapoo Shores Waterfront Cottages
1,205,Seinäjoki,0,FI,1624610705,60,62.7945,22.8282,76.10,1.01,Hotelli OnnenTähti
2,261,Chibuto,0,MZ,1624610739,36,-24.6867,33.5306,76.59,6.87,Go Wild Beach Resort
3,510,Fes,0,MA,1624610890,50,34.0372,-4.9998,79.05,6.91,


In [11]:
hotel_df.head()

,City ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
0,143,Walker,0,US,1624610661,95,30.7335,-95.5669,78.53,5.50,Kickapoo Shores Waterfront Cottages
1,205,Seinäjoki,0,FI,1624610705,60,62.7945,22.8282,76.10,1.01,Hotelli OnnenTähti
2,261,Chibuto,0,MZ,1624610739,36,-24.6867,33.5306,76.59,6.87,Go Wild Beach Resort
3,510,Fes,0,MA,1624610890,50,34.0372,-4.9998,79.05,6.91,


In [12]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [13]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations,
    info_box_content=[f"Hotel loc: {rate}" for rate in hotel_info])

fig.add_layer(markers)

# Display Map
fig


Figure(layout=FigureLayout(height='420px'))